In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# for rohit to run and connect to files
%cd /content/drive/My Drive

# for nathan to run and connect to files
# %cd /content/drive/My Drive/Gatech/Current Classes/CS7643

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [ ]:
%cd FinalProject/data/preprocessed_data

/content/drive/.shortcut-targets-by-id/1-5I_iGwBvybVEHxGQIAMG09aa4G4xdMy/FinalProject/data/preprocessed_data


In [ ]:
%ls

cnn_lstm.pth  test/  train/


In [ ]:
import os
from PIL import Image
import torch
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
# torch.set_default_dtype()
torch.set_default_device("cuda")
from torch.utils.data import Dataset, DataLoader
import tqdm
import torch.nn as nn
import torchvision.models as models
import pickle
import zipfile
import torchvision.transforms as transforms
import re
import random
import numpy as np

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, input_folder, output_folder, transform=None):
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.transform = transform
        self.data = self._load_data()

    def _load_data(self):
        input_files = sorted(os.listdir(self.input_folder))
        output_files = sorted(os.listdir(self.output_folder))

        data = []
        for img_file, txt_file in zip(input_files, output_files):
            if img_file.endswith('.png') and txt_file.endswith('.txt'):
                img_path = os.path.join(self.input_folder, img_file)
                txt_path = os.path.join(self.output_folder, txt_file)

                with open(txt_path, 'r') as file:
                    text_content = file.read()

                data.append({'image': img_path, 'text_content': text_content})

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]['image']
        #image = Image.open(image)
        text_content = self.data[idx]['text_content']

        #if self.transform:
        #    image = self.transform(image)

        return {'image': image, 'text_content': text_content}

In [ ]:
class CustomDatasetLimited(Dataset):
    def __init__(self, input_folder, output_folder, lim):
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.transform = transform
        self.lim = lim
        self.data = self._load_data()


    def _load_data(self):
        if self.lim is None:
          self.lim = 1600
        input_files = sorted(os.listdir(self.input_folder))
        output_files = sorted(os.listdir(self.output_folder))

        paired_files = list(zip(input_files, output_files))
        random.shuffle(paired_files)
        input_files, output_files = zip(*paired_files)

        data = []
        count = 0
        for img_file, txt_file in zip(input_files, output_files):
            if img_file.endswith('.png') and txt_file.endswith('.txt'):
                img_path = os.path.join(self.input_folder, img_file)
                txt_path = os.path.join(self.output_folder, txt_file)

                with open(txt_path, 'r') as file:
                    text_content = file.read()

                data.append({'image': img_path, 'text_content': text_content})
                count += 1
            if count >= self.lim:
              return data

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]['image']
        #image = Image.open(image)
        text_content = self.data[idx]['text_content']

        #if self.transform:
        #    image = self.transform(image)

        return {'image': image, 'text_content': text_content}

In [ ]:

# input_folder_path = 'train/IMG'
# output_folder_path = 'train/INKML'

# ## Need to set transform to none right now, I did not implement any Image.open in the __getitem
# dataset = CustomDataset(input_folder_path, output_folder_path, transform=None)

# batch_size = 32
# data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# torch.save(dataset, 'train_dataset.pth')

# Test Data_loader load in

In [ ]:
transform = transforms.Compose([
    transforms.Resize((1010, 1010)),
    transforms.ToTensor()
])

In [ ]:
# dataset = torch.load('train_dataset.pth')

# batch_size = 32
# data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# for i, batch in enumerate(data_loader):
#     print(batch['text_content'][0])
#     img = Image.open(batch['image'][0])
#     img = transform(img)
#     img = transforms.ToPILImage()(img)
#     img.show()
#     if i == 0:
#         break


In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self, encoded_image_size):
        super(EncoderCNN, self).__init__()
        self.cnn = models.resnet50(pretrained=True)
        modules = list(self.cnn.children())[:-2]
        self.cnn = nn.Sequential(*modules)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((encoded_image_size, encoded_image_size))

    def forward(self, images):
        features = self.cnn(images)
        features = self.adaptive_pool(features)
        return features.permute(0, 2, 3, 1)

class DecoderTransformer(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=6, num_heads=8, dropout=0.1):
        super(DecoderTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(embed_size)
        decoder_layer = nn.TransformerDecoderLayer(embed_size, num_heads, hidden_size, dropout)
        self.transformer = nn.TransformerDecoder(decoder_layer, num_layers)
        self.linear = nn.Linear(embed_size, vocab_size)
        self.embed_size = embed_size

    def forward(self, images, captions):
        embeddings = self.embedding(captions)
        positional_encoding = self.positional_encoding(embeddings)
        embeddings = embeddings + positional_encoding
        features = images
        batch_size, height, width, channels = features.size()
        features = features.reshape(batch_size, height*width, channels)
        linear_projection = nn.Linear(channels, 256)
        features = linear_projection(features)
        features = features.permute(1, 0, 2)
        print(features.shape, embeddings.shape)

        outputs = self.transformer(features, embeddings)
        outputs = self.linear(outputs)
        return outputs

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.positional_encoding = torch.zeros(max_len, d_model)
        self.positional_encoding[:, 0::2] = torch.sin(position * div_term)
        self.positional_encoding[:, 1::2] = torch.cos(position * div_term)
        self.positional_encoding = self.positional_encoding.unsqueeze(0)

    def forward(self, x):
        return self.positional_encoding[:, :x.size(1)]

In [ ]:
input_folder_path = 'train/IMG'
output_folder_path = 'train/INKML'
dataset = CustomDatasetLimited(input_folder_path, output_folder_path, lim=1600)
batch_size = 16
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
print(len(data_loader))
print(len(dataset))

100
1600


In [ ]:
tokens = []
word_to_idx = {}
token_pattern = r'\\[a-zA-Z]+|\\[^a-zA-Z0-9\s]|\{[^\{\}]*\}|\S+'
idx = 0

for i, batch in enumerate(data_loader):
    annotation = batch['text_content'][0]
    words = re.findall(token_pattern, annotation)
    for word in words:
        if word not in word_to_idx:
            word_to_idx[idx] = word
            idx += 1

    print(str(i) + ": ")
    print(words)
    tokens.extend(words)

unique_tokens = set(tokens)

## Plus 1 for unknown token
vocab_size = len(unique_tokens) + 4


word_to_idx['BOS'] = idx
word_to_idx['EOS'] = idx + 1
word_to_idx['PAD'] = idx + 2
word_to_idx['UNK'] = idx + 3

print(vocab_size)

0: 
['\\frac', '{', '-', 'b', '-', '\\sqrt', '{ b ^ 2 - 4 a c }', '}', '{ 2 a }']
1: 
['$', '\\sum', '\\limits', '^', '{1}', '_', '{k = 1}', 'a', '_', '{k}', '=', 'a', '_', '{1}', '$']
2: 
['$x', '=', 'r', '\\cos', '\\theta', '$']
3: 
['$3a', '+', '2', '\\leq', '5$']
4: 
['-6\\cos', '\\theta']
5: 
['$(s,t)$']
6: 
['A', '=', '\\sqrt', '{a', '+', '\\frac', '{1}', '{\\sqrt{a', '+', '\\frac', '{1}', '{\\sqrt{a}}}}}', '+', '\\sqrt', '{b}']
7: 
['$(-a)^{n}', '=', '(-a)\\times(-a)\\times(-a)\\times', '\\ldots', '\\times', '(-a)$']
8: 
['x_x^x', '+', 'y_y^y', '+', 'z_z^z', '-', 'x', '-', 'y', '-', 'z']
9: 
['e_', '{', 'f_', '{ g_h }', '}']
10: 
['x_{12}x_{34}=x_{34}x_{12}']
11: 
['$n_1\\sin', 'i_1', '=n_2', '\\sin', 'i_2$']
12: 
['$2', '\\times', '3$']
13: 
['a', '(', 'n', ')', '=', '\\sum', '_{k', '=', '1}^n', '(', '-', '1', ')^{n', '-', 'k}', 'k', '!']
14: 
['[a_1]', '\\times', '[b_2]', '\\times', '[a_3]']
15: 
['$\\frac', '{V}', '{T}', '=', '\\frac', '{nR}', '{p}', '$']
16: 
['x_{i}', '-', 

In [ ]:
print(word_to_idx)

{'\\frac': 0, '{': 1, '-': 2, 'b': 3, '\\sqrt': 4, '{ b ^ 2 - 4 a c }': 5, '}': 6, '{ 2 a }': 7, '$': 8, '\\sum': 9, '\\limits': 10, '^': 11, '{1}': 12, '_': 13, '{k = 1}': 14, 'a': 15, '{k}': 16, '=': 17, '$x': 18, 'r': 19, '\\cos': 20, '\\theta': 21, '$3a': 22, '+': 23, '2': 24, '\\leq': 25, '5$': 26, '-6\\cos': 27, '$(s,t)$': 28, 'A': 29, '{a': 30, '{\\sqrt{a': 31, '{\\sqrt{a}}}}}': 32, '{b}': 33, '$(-a)^{n}': 34, '(-a)\\times(-a)\\times(-a)\\times': 35, '\\ldots': 36, '\\times': 37, '(-a)$': 38, 'x_x^x': 39, 'y_y^y': 40, 'z_z^z': 41, 'x': 42, 'y': 43, 'z': 44, 'e_': 45, 'f_': 46, '{ g_h }': 47, 'x_{12}x_{34}=x_{34}x_{12}': 48, '$n_1\\sin': 49, 'i_1': 50, '=n_2': 51, '\\sin': 52, 'i_2$': 53, '$2': 54, '3$': 55, '(': 56, 'n': 57, ')': 58, '_{k': 59, '1}^n': 60, '1': 61, ')^{n': 62, 'k}': 63, 'k': 64, '!': 65, '[a_1]': 66, '[b_2]': 67, '[a_3]': 68, '$\\frac': 69, '{V}': 70, '{T}': 71, '{nR}': 72, '{p}': 73, 'x_{i}': 74, 'x_{i': 75, '1}': 76, '2}': 77, '(ab)=\\frac{1}{2}(ab+ba)': 78, '

In [ ]:
unk_token = len(word_to_idx) - 1
pad_token = len(word_to_idx) - 2
eos_token = len(word_to_idx) - 3
bos_token = len(word_to_idx) - 4

In [ ]:
print(unk_token, pad_token, eos_token, bos_token)

269 268 267 266


In [ ]:
class ImageToLatex(nn.Module):
    def __init__(self, encoded_image_size, embed_size, hidden_size, vocab_size, num_layers=6, num_heads=8, dropout=0.1):
        super(ImageToLatex, self).__init__()
        self.encoder = EncoderCNN(encoded_image_size)
        self.decoder = DecoderTransformer(embed_size, hidden_size, vocab_size, num_layers, num_heads, dropout)

    def forward(self, images, captions):
        features = self.encoder(images)
        features = features.permute(0, 2, 1)
        print(features.shape, captions.shape)
        outputs = self.decoder(features, captions)
        return outputs


# model = ImageToLatex(encoded_image_size=64, embed_size=256, hidden_size=512, vocab_size=vocab_size, num_layers=6, num_heads=8, dropout=0.1)


In [ ]:
def tokenize(caption):
  token_pattern = r'\\[a-zA-Z]+|\\[^a-zA-Z0-9\s]|\{[^\{\}]*\}|\S+'
  words = re.findall(token_pattern, caption)
  res = []
  for word in words:
    if word not in word_to_idx:
      res.append(word_to_idx['UNK'])
    else:
      res.append(word_to_idx[word])
  return res

# Transformer model with Multi-head Attention
Something is off with the shapes in the decoder layer --> It causes some sort of memory leak.

In [ ]:
# EPOCHS = 20
# optimizer = torch.optim.AdamW(model.parameters(), lr=1.0E-5)
# criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# losses = []
# for epoch in range(EPOCHS):
#     model.train()
#     for c, batch in tqdm.tqdm(enumerate(data_loader)):
#         img = batch["image"]
#         processed_images = []

#         # Process each image in the batch
#         for image_path in img:
#             pil_image = Image.open(image_path).convert('RGB')
#             transformed_image = transform(pil_image)
#             processed_images.append(transformed_image)

#         batch_tensor = torch.stack(processed_images)

#         ## TODO: Rightshift the targets and also add the BOS token
#         annotations = batch["text_content"]
#         latex = [tokenize(caption) for caption in annotations]
#         max_caption_length = max(len(caption) for caption in latex)
#         padded_captions = [caption + [pad_token] * (max_caption_length - len(caption)) for caption in latex]
#         padded_captions = [bos_token] + padded_captions[:-1]
#         captions_tensor = torch.tensor(padded_captions)

#         optimizer.zero_grad()
#         logits = model(batch_tensor, captions_tensor)
#         print("Starting Criterion")
#         loss = criterion(logits, captions_tensor)
#         print(loss)
#         loss.backward()
#         losses.append(loss.item())
#         optimizer.step()

# OR Just CNN with LSTM layer


CNN_LSTM Model runs correctly

In [ ]:
class CNN_LSTM(nn.Module):
    def __init__(self, lstm_hidden_size, num_classes, resnet_pretrained=True):
        super(CNN_LSTM, self).__init__()

        self.resnet = models.resnet50(pretrained=resnet_pretrained)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])  # Remove last FC layer and avg pool

        self.lstm = nn.LSTM(2048, lstm_hidden_size, batch_first=True)

        self.fc = nn.Linear(lstm_hidden_size, num_classes)

    def forward(self, x):
        with torch.no_grad():
            features = self.resnet(x)
        features = features.view(x.size(0), -1, features.size(1))

        lstm_out, _ = self.lstm(features)

        lstm_out = lstm_out[:, -1, :]  # Consider only the output of the last time step

        output = self.fc(lstm_out)
        return output

In [ ]:
lstm_hidden_size = 256

# Create an instance of the model
model = CNN_LSTM(lstm_hidden_size, vocab_size)
model.to("cuda")

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define loss and optimizer
EPOCHS = 5

# criterion = nn.CrossEntropyLoss()
# criterion = nn.functional.cross_entropy()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
losses = []

for epoch in range(EPOCHS):
    for idx, batch in tqdm.tqdm(enumerate(data_loader)):
        optimizer.zero_grad()

        img = batch["image"]
        processed_images = []

        # Process each image in the batch
        for image_path in img:
            pil_image = Image.open(image_path).convert('RGB')
            transformed_image = transform(pil_image)
            processed_images.append(transformed_image)

        batch_images = torch.stack(processed_images)
        batch_images = batch_images.to("cuda")

        annotations = batch["text_content"]
        latex = [tokenize(caption) for caption in annotations]
        max_caption_length = max(len(caption) for caption in latex)
        padded_captions = [caption + [pad_token] * (max_caption_length - len(caption)) for caption in latex]
        padded_captions = [[bos_token] + batch[:-1] for batch in padded_captions]
        captions_tensor = torch.tensor(padded_captions)

        print("Starting forward")
        outputs = model(batch_images)
        print("Starting criterion")
        outputs = outputs.repeat(max_caption_length, 1)
        outputs = outputs.view(-1, vocab_size)

        mask = (captions_tensor.view(-1) != pad_token).float()

        loss = nn.functional.cross_entropy(outputs, captions_tensor.view(-1), reduction='none')

        loss = loss * mask
        loss = torch.sum(loss) / torch.sum(mask)

        losses.append(loss)
        print(loss)

        loss.backward()
        optimizer.step()

0it [00:00, ?it/s]

Starting forward


1it [00:01,  1.60s/it]

Starting criterion
tensor(4.3441, grad_fn=<DivBackward0>)
Starting forward


2it [00:03,  1.55s/it]

...

Starting criterion
tensor(2.7999, grad_fn=<DivBackward0>)
Starting forward


99it [02:31,  1.50s/it]

Starting criterion
tensor(2.9557, grad_fn=<DivBackward0>)
Starting forward


100it [02:33,  1.53s/it]

Starting criterion
tensor(3.1617, grad_fn=<DivBackward0>)


# Save our model

In [ ]:
model_path = 'cnn_lstm.pth'
torch.save(model.state_dict(), model_path)

# Testing our Model


In [ ]:
# insert testing code here lols

In [ ]:
import pandas as pd
import random
import mysql.connector

def connect_to_mysql():
    try:
        conn = mysql.connector.connect(
            host='your_host',
            user='your_username',
            password='your_password',
            database='aeurocast'
        )
        return conn
    except mysql.connector.Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

# Function to loop through CSV files, sample rows, and insert into MySQL table
def sample_and_insert_to_mysql(folder_path):
    conn = connect_to_mysql()
    if conn is None:
        return

    cursor = conn.cursor()

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            try:
                df = pd.read_csv(file_path)
                sampled_rows = df.sample(n=1000, replace=False)  # Randomly sample 1000 rows

                for index, row in sampled_rows.iterrows():
                    sql = "INSERT INTO FlightInformation (Plug in the column names comma separated) VALUES (%s, %s, match types)"  # Replace column names
                    values = tuple(row)  # Assuming the columns in CSV match the table columns, make sure the ordering is right
                    cursor.execute(sql, values)

                conn.commit()
                print(f"Inserted 1000 rows from {file_path} to MySQL table.")
            except Exception as e:
                conn.rollback()
                print(f"Error: {e} occurred while processing {file_path}. Rolling back changes.")

    cursor.close()
    conn.close()

# List of file paths (replace with your file paths)
folder_path = ""

# Call function to sample rows and insert into MySQL table
sample_and_insert_to_mysql(folder_path)